In [1]:
 from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd gdrive/My\ Drive

/content/gdrive/My Drive


In [3]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [4]:
trainDataGen = ImageDataGenerator(rescale = 1./255)
valDataGen = ImageDataGenerator(rescale = 1./255)
testDataGen = ImageDataGenerator(rescale = 1./255 )


In [5]:
baseDir = '/content/gdrive/My Drive/Colab Notebooks/CNN/dataset'

trainDir = os.path.join(baseDir,'train')
validationDir = os.path.join(baseDir,'validation')
testDir = os.path.join(baseDir,'test')


In [6]:
trainGenerator = trainDataGen.flow_from_directory(trainDir, target_size=(224,224), batch_size = 20, class_mode='binary')
validationGenerator = valDataGen.flow_from_directory(validationDir, target_size=(224,224), batch_size = 20, class_mode='binary')
testGenerator = testDataGen.flow_from_directory(testDir, target_size=(224,224), batch_size = 20, class_mode='binary')


Found 335 images belonging to 2 classes.
Found 54 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [7]:
from keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras import models
from keras import layers

In [9]:
VGGNet = VGG16()
#VGGNet.summary()

for layer in VGGNet.layers:
  layer.trainable = False # 설계된 vgg16 모델의 w,b를 업데이트 안 하겠다
  
vgg_maxpool5 = VGGNet.get_layer('block5_pool').output

FeatureFlatten = Flatten()(vgg_maxpool5)
dense = Dense(10, name = 'dense',activation='relu')(FeatureFlatten)
predictions = Dense(1,activation = 'sigmoid')(dense)
New_VGGmodel = Model(inputs = VGGNet.input, outputs = predictions)
New_VGGmodel.compile(optimizer = 'rmsprop',loss='binary_crossentropy', metrics = ['accuracy'])

New_VGGmodel.summary()

Model: "functional_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [10]:
history = New_VGGmodel.fit(trainGenerator, steps_per_epoch=10, epochs=30, validation_data=validationGenerator)

Epoch 1/30
10/10 [==============================] - 6s 574ms/step - loss: 1.1319 - accuracy: 0.5200 - val_loss: 0.6190 - val_accuracy: 0.5000
Epoch 2/30
10/10 [==============================] - 5s 479ms/step - loss: 0.6378 - accuracy: 0.5700 - val_loss: 0.4858 - val_accuracy: 0.9074
Epoch 3/30
10/10 [==============================] - 4s 444ms/step - loss: 0.4862 - accuracy: 0.7700 - val_loss: 0.5327 - val_accuracy: 0.6481
Epoch 4/30
10/10 [==============================] - 4s 445ms/step - loss: 0.4374 - accuracy: 0.8050 - val_loss: 0.4322 - val_accuracy: 0.7407
Epoch 5/30
10/10 [==============================] - 5s 495ms/step - loss: 0.3700 - accuracy: 0.8667 - val_loss: 0.4159 - val_accuracy: 0.7593
Epoch 6/30
10/10 [==============================] - 4s 396ms/step - loss: 0.3248 - accuracy: 0.8600 - val_loss: 0.3181 - val_accuracy: 0.8889
Epoch 7/30
10/10 [==============================] - 4s 431ms/step - loss: 0.2111 - accuracy: 0.9692 - val_loss: 0.2975 - val_accuracy: 0.8704
Epoch 

In [11]:
testSet = testDataGen.flow_from_directory(testDir, target_size = (224,224,), batch_size =32, class_mode = 'binary')
output = New_VGGmodel.predict(testSet)
print(testSet.class_indices)
print(testSet.filenames)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(output)

Found 16 images belonging to 2 classes.
{'cats': 0, 'dogs': 1}
['cats/c1.jpg', 'cats/c2.JPG', 'cats/c3.jpg', 'cats/c4.jpg', 'cats/c5.jpg', 'cats/c6.jpg', 'cats/c7.jpg', 'cats/c8.jpg', 'dogs/d1.jpg', 'dogs/d2.png', 'dogs/d3.jpg', 'dogs/d4.jpg', 'dogs/d5.jpg', 'dogs/d6.jpg', 'dogs/d7.jpg', 'dogs/d8.png']
[[0.562]
 [1.000]
 [0.000]
 [0.001]
 [0.153]
 [0.004]
 [0.000]
 [0.000]
 [0.995]
 [0.000]
 [0.000]
 [1.000]
 [0.533]
 [1.000]
 [0.944]
 [1.000]]


In [12]:
loss, acc = New_VGGmodel.evaluate(testSet,verbose=0)
print(loss)
print(acc)

0.08996438980102539
1.0
